# Molecule Visualization with nglview

Visualize PDB / GRO / MOL2 files produced by the simulation pipeline.

In [2]:
import nglview as nv
from pathlib import Path

## Molecule visualization

In [2]:
# Path to the PDB (or GRO / MOL2) file you want to visualise
pdb_path = "/home/shpark/prj-amd/data/molecule/chignolin/temp.pdb"   # ← change as needed

assert Path(pdb_path).exists(), f"File not found: {pdb_path}"
print(f"Loading: {pdb_path}")

Loading: /home/shpark/prj-amd/data/molecule/chignolin/temp.pdb


In [3]:
view = nv.show_file(pdb_path)

# Representations
view.clear_representations()
view.add_representation("cartoon", selection="protein", color="residueindex")
# view.add_representation("surface", selection="protein", color="element", opacity=0.1)
# view.add_representation("ball+stick", selection="water", color="element", opacity=0.3)

view.parameters = {"backgroundColor": "white"}
view.camera = "perspective"
view

NGLWidget()

In [4]:
view2 = nv.show_file(pdb_path)
view2.clear_representations()
view2.add_representation("stick", color="element")
# view2.add_representation("catoon")
# view2.parameters = {"backgroundColor": "white"}
view2

NGLWidget()

## Trajectory Visualization

Load a GROMACS trajectory (`.xtc` / `.trr`) + topology (`.gro` / `.pdb`) and explore it interactively with nglview.

Requires **mdtraj** (`pip install mdtraj`).

In [3]:
import mdtraj as md
import nglview as nv

# ── Paths ─────────────────────────────────────────────────────────────────────
session_root = "/home/shpark/prj-amd/outputs/debug/0228-214213"
topo_path = f"{session_root}/data/temp_box.gro"    # topology / reference coordinates
traj_path = f"{session_root}/data/simulation/md.xtc"  # trajectory

assert Path(topo_path).exists(), f"Topology not found: {topo_path}"
assert Path(traj_path).exists(), f"Trajectory not found: {traj_path}"
print(f"Topology : {topo_path}")
print(f"Trajectory: {traj_path}")

Topology : /home/shpark/prj-amd/outputs/debug/0228-214213/data/temp_box.gro
Trajectory: /home/shpark/prj-amd/outputs/debug/0228-214213/data/simulation/md.xtc


In [4]:
# Load trajectory with mdtraj (strip solvent to speed up rendering)
traj = md.load(traj_path, top=topo_path)
print(f"Frames : {traj.n_frames}")
print(f"Atoms  : {traj.n_atoms}")
print(f"Residues: {traj.n_residues}")

# Optional: superpose all frames onto the first to remove overall translation/rotation
traj.superpose(traj, 0)

# Downsample if very long (keep every N-th frame for interactive performance)
stride = max(1, traj.n_frames // 500)
if stride > 1:
    traj = traj[::stride]
    print(f"Downsampled to {traj.n_frames} frames (stride={stride})")

Frames : 501
Atoms  : 166
Residues: 10


In [5]:
traj

<mdtraj.Trajectory with 501 frames, 166 atoms, 10 residues, and unitcells at 0x7f1bbe10ad10>

In [6]:
# ── Trajectory viewer ─────────────────────────────────────────────────────────
# Build the widget explicitly so the trajectory player stays properly connected.
# nv.show_mdtraj() + clear_representations() in the same cell can race against
# widget init and freeze on frame 0.
traj_view = nv.NGLWidget()
traj_view.add_trajectory(nv.MDTrajTrajectory(traj))

# Representations (applied after trajectory component is registered)
traj_view.clear_representations()
traj_view.add_representation("ball+stick", color="element")
# traj_view.add_representation("cartoon", selection="protein", color="residueindex")
# traj_view.add_representation("surface", color="white", opacity=0.15)

# Player speed: delay in ms per frame
# traj_view.player.parameters = dict(delay=80, step=1)

traj_view

NGLWidget(max_frame=500)

In [10]:
view = nv.show_mdtraj(traj)
view.add_representation("cartoon", selection="protein", color="residueindex")
# view.add_representation("surface", color="white", opacity=0.15)
# view.parameters = {"backgroundColor": "black"}


In [11]:
view

NGLWidget(max_frame=500)